# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [84]:
# import libraries
import pandas as pd
import numpy as np
import pickle

from sqlalchemy import create_engine

import re

import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
# load data from database
engine = create_engine('sqlite:///Project.db')
df = pd.read_sql_table("Messages",engine)
X = df['message']
Y = df[df[df.columns[df.columns!='child_alone']].columns[4:]]
Y.sum(axis=0)

related                   20252
request                    4464
offer                       118
aid_related               10841
medical_help               2081
medical_products           1311
search_and_rescue           724
security                    471
military                    859
water                      1669
food                       2917
shelter                    2308
clothing                    404
money                       603
missing_people              298
refugees                    874
death                      1192
other_aid                  3441
infrastructure_related     1705
transport                  1199
buildings                  1331
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7286
floods                     2149
storm                      2440
fire                        282
earthqua

### 2. Write a tokenization function to process your text data

In [65]:
def tokenize(text):
    
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]

    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]    
    tokens = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [66]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [67]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [68]:
Y_pred = pipeline.predict(X_test)

In [70]:
for i in range(Y_pred.shape[1]):
    print(classification_report(np.array(Y_test)[:,i], np.array(Y_pred)[:,i]))

             precision    recall  f1-score   support

          0       0.73      0.43      0.54      1545
          1       0.84      0.95      0.89      4953
          2       0.00      0.00      0.00        47

avg / total       0.80      0.82      0.80      6545

             precision    recall  f1-score   support

          0       0.92      0.98      0.95      5468
          1       0.83      0.56      0.67      1077

avg / total       0.90      0.91      0.90      6545

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3852
          1       0.77      0.67      0.72      2693

avg / total       0.78      0.78      0.78      6545

             precision    recall  f1-score   support

          0       0.93      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [79]:
parameters = {'clf__estimator__solver':['lbfgs', 'liblinear', 'newton-cg'],
             'clf__estimator__C':[1.,.5,.25]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__solver': ['lbfgs', 'liblinear', 'newton-cg'], 'clf__estimator__C': [1.0, 0.5, 0.25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [80]:
estimator = cv.best_estimator_

Y_pred = estimator.predict(X_test)

for i in range(Y_pred.shape[1]):
    print(classification_report(np.array(Y_test)[:,i], np.array(Y_pred)[:,i]))

             precision    recall  f1-score   support

          0       0.73      0.43      0.54      1545
          1       0.84      0.95      0.89      4953
          2       0.00      0.00      0.00        47

avg / total       0.80      0.82      0.80      6545

             precision    recall  f1-score   support

          0       0.92      0.98      0.95      5468
          1       0.83      0.56      0.67      1077

avg / total       0.90      0.91      0.90      6545

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3852
          1       0.77      0.67      0.72      2693

avg / total       0.78      0.78      0.78      6545

             precision    recall  f1-score   support

          0       0.93      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
cv.best_params_

{'clf__estimator__C': 1.0, 'clf__estimator__solver': 'lbfgs'}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [82]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SVC()))
    ])

pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_test)

for i in range(Y_pred.shape[1]):
    print(classification_report(np.array(Y_test)[:,i], np.array(Y_pred)[:,i]))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1545
          1       0.76      1.00      0.86      4953
          2       0.00      0.00      0.00        47

avg / total       0.57      0.76      0.65      6545

             precision    recall  f1-score   support

          0       0.84      1.00      0.91      5468
          1       0.00      0.00      0.00      1077

avg / total       0.70      0.84      0.76      6545

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

             precision    recall  f1-score   support

          0       0.59      1.00      0.74      3852
          1       0.00      0.00      0.00      2693

avg / total       0.35      0.59      0.44      6545

             precision    recall  f1-score   support

          0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [83]:
# open a file, where you ant to store the data
file = open('model', 'wb')

# dump information to that file
pickle.dump(estimator, file)

# close the file
file.close()

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.